In [1]:
import torch
import numpy as np
import pandas as pd
from torch import nn
from sklearn.metrics import r2_score
from mlp_explainer.researcher import Researcher
from torch.utils.data import DataLoader, Dataset
from mlp_explainer.build_model import BuildModel
from mlp_explainer.synthetic_data import SyntheticData

In [2]:
identifier = "experiment_1"
experiment_1 = Researcher(identifier)

In [3]:
experiment_1.init()

In [4]:
data = pd.read_csv('data/experiment_1_dataset.csv'); data

,X_1,X_2,X_4,X_3,X_0
0,-0.656031,0.344900,-1.334576,0.468296,-0.809016
1,-2.251120,-2.186524,-0.857358,0.048205,0.205389
2,0.302455,-0.873230,0.335856,-0.487675,1.552792
3,1.867108,0.480192,2.070642,-1.714295,5.495954
4,-0.250600,-0.364680,0.895084,-0.994032,3.670706
...,...,...,...,...,...
123,1.847180,0.978568,0.342745,-0.648070,1.377007
124,-0.015760,1.847228,0.278737,-0.615867,1.246472
125,0.342962,-0.498526,0.615678,-0.844891,1.598670
126,-0.977038,-1.269541,-1.278382,0.545446,-0.930721


In [22]:
class MLP(nn.Module):
    def __init__(self, num_features):

        super().__init__()
        self.first = nn.Linear(num_features, 8, True)
        self.last = nn.Linear(8, 1, True)

    def forward(self, input_):

        out = self.last(self.first(input_))
        return out

In [23]:
model = MLP(data.shape[1] - 1)

In [24]:
build_model = BuildModel(model, data, identifier)

In [28]:
build_model.training_loop(
    num_epochs = 25, 
    batch_size = len(build_model.train_data), 
    learning_rate = 1e-2, 
    device = "cpu"
)

Epoch 1/25 | MSE: 1.9685
Validation MSE: 1.2799 | Validation R²: -0.0387
Epoch 2/25 | MSE: 1.9685
Validation MSE: 1.2799 | Validation R²: -0.0388
Epoch 3/25 | MSE: 1.9685
Validation MSE: 1.2799 | Validation R²: -0.0389
Epoch 4/25 | MSE: 1.9685
Validation MSE: 1.2799 | Validation R²: -0.0389
Epoch 5/25 | MSE: 1.9685
Validation MSE: 1.2800 | Validation R²: -0.0390
Epoch 6/25 | MSE: 1.9685
Validation MSE: 1.2800 | Validation R²: -0.0390
Epoch 7/25 | MSE: 1.9685
Validation MSE: 1.2800 | Validation R²: -0.0391
Epoch 8/25 | MSE: 1.9685
Validation MSE: 1.2800 | Validation R²: -0.0391
Epoch 9/25 | MSE: 1.9685
Validation MSE: 1.2800 | Validation R²: -0.0392
Epoch 10/25 | MSE: 1.9685
Validation MSE: 1.2800 | Validation R²: -0.0392
Epoch 11/25 | MSE: 1.9685
Validation MSE: 1.2800 | Validation R²: -0.0393
Epoch 12/25 | MSE: 1.9685
Validation MSE: 1.2801 | Validation R²: -0.0393
Epoch 13/25 | MSE: 1.9685
Validation MSE: 1.2801 | Validation R²: -0.0394
Epoch 14/25 | MSE: 1.9685
Validation MSE: 1.280

/home/vmuser/anaconda3/lib/python3.12/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([76])) that is different to the input size (torch.Size([76, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/vmuser/anaconda3/lib/python3.12/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([26])) that is different to the input size (torch.Size([26, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/vmuser/anaconda3/lib/python3.12/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([76])) that is different to the input size (torch.Size([76, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the s